#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime
import os


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)


LOG_FILENAME = 'log.csv'
ARCHIVE_FOLDER = 'archive'
LOG_RECORDS_LIMIT = 10

current_records = 0


# Добавление строки в лог
def add_log(sname):
    global current_records
    if current_records < LOG_RECORDS_LIMIT:
        append_to_log(LOG_FILENAME, create_log_row(sname))
        current_records += 1
    else:
        save_file_to_archive(LOG_FILENAME, ARCHIVE_FOLDER, datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '_log.csv')
        create_file(LOG_FILENAME)
        append_to_log(LOG_FILENAME, create_log_row(sname))
        current_records = 1
server.register_function(add_log, 'add_log')


def get_log(from_date, to_date, op_type = None):
    from_date = datetime.datetime.strptime(from_date, '%Y-%m-%d %H:%M:%S')
    to_date = datetime.datetime.strptime(to_date, '%Y-%m-%d %H:%M:%S')
    result = []
    log_files = get_files(ARCHIVE_FOLDER)
    log_files.append(LOG_FILENAME)
    
    print(from_date.strftime("%Y%m%d"))
    
    for log_file in log_files:
        if (log_file == LOG_FILENAME) or (from_date <= date_from_filename(log_file) <= to_date):
            file = open(log_file, 'r')
            lines = file.readlines()
            for line in lines:
                line_tokens = line.split(",")
                type = line_tokens[0]
                date = datetime.datetime.strptime(line_tokens[1].removesuffix("\n"), '%Y-%m-%d %H:%M:%S')
                if (from_date <= date <= to_date) and (op_type is None or op_type == type):
                    result.append(line)
    return result
server.register_function(get_log, 'get_log')


def date_from_filename(filename):
    return datetime.datetime.strptime(filename.replace('_log.csv', '').replace(ARCHIVE_FOLDER + os.sep, ''), '%Y%m%d_%H%M%S')


def save_file_to_archive(filename, folder, new_filename):
    os.replace(filename, folder + os.sep + new_filename)


def create_file(filename):
    open(filename, 'w').close()


def create_log_row(sname):
    return str(sname) + ',' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n'


def append_to_log(filename, filerow):
    f = open(filename,'a')
    f.write(filerow)
    f.close()
    
def get_files(folder):
    result = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            result.append(folder + os.sep + file)
    return result
    

print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...
20230101


127.0.0.1 - - [10/Oct/2023 23:44:50] "POST /RPC2 HTTP/1.1" 200 -


20230101


127.0.0.1 - - [10/Oct/2023 23:44:52] "POST /RPC2 HTTP/1.1" 200 -


20230101


127.0.0.1 - - [10/Oct/2023 23:45:22] "POST /RPC2 HTTP/1.1" 200 -


20230101


127.0.0.1 - - [10/Oct/2023 23:45:24] "POST /RPC2 HTTP/1.1" 200 -
